# Neural ChatBot

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

## 1. Load and Preprocess Data

In [2]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\r\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\r\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\r\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\r\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\r\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\r\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\r\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\r\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\r\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\r\n'


## 2. Create Formatted Data File
    query sentence <tab> response sentece

In [3]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            #Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines

# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            lineIds = eval(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations

# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [4]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

## 3. Load and Trim Data ( Preprocess Data )

In [5]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
    
    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), 
            len(self.word2index), 
            len(keep_words) / len(self.word2index)
        ))
        
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [6]:
MAX_LENGTH = 10      #Maximum sentence length to consider

def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [7]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


## 4. Prepare data for models

In [8]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  50,  124,  424,  122,   77],
        [  94,  601,  174,    7,    6],
        [   7,    4,   53,  197,    2],
        [ 141,  147,  217,  117,    0],
        [  83,  582,    4,    6,    0],
        [  75,   68,    2,    2,    0],
        [1762,   76,    0,    0,    0],
        [1763,  218,    0,    0,    0],
        [   6,    6,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([10, 10,  6,  6,  3])
target_variable: tensor([[  33,   68,   50,   25,  424],
        [   4,   50,    6, 1187,   83],
        [   4,  218,    2,   25,   36],
        [   4,    6,    0,  354,  601],
        [   2,    2,    0,  283,    4],
        [   0,    0,    0,   18,   70],
        [   0,    0,    0,    7,  862],
        [   0,    0,    0,    4,    4],
        [   0,    0,    0,    2,    2]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0,

## 5. Define Model

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [10]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [11]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## 6. Define Training procedure

In [12]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [13]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [14]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [15]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## 7. Evaluate my text

In [16]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## 7. Run Model

In [17]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [18]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 4.0
n_iteration = 10000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9560
Iteration: 2; Percent complete: 0.0%; Average loss: 8.8577
Iteration: 3; Percent complete: 0.0%; Average loss: 8.7208
Iteration: 4; Percent complete: 0.0%; Average loss: 8.4805
Iteration: 5; Percent complete: 0.1%; Average loss: 8.1181
Iteration: 6; Percent complete: 0.1%; Average loss: 7.7597
Iteration: 7; Percent complete: 0.1%; Average loss: 7.1938
Iteration: 8; Percent complete: 0.1%; Average loss: 7.0275
Iteration: 9; Percent complete: 0.1%; Average loss: 7.0025
Iteration: 10; Percent complete: 0.1%; Average loss: 6.8999
Iteration: 11; Percent complete: 0.1%; Average loss: 6.7938
Iteration: 12; Percent complete: 0.1%; Average loss: 6.2504
Iteration: 13; Percent complete: 0.1%; Average loss: 5.8487
Iteration: 14; Percent complete: 0.1%; Average loss: 5.8916
Iteration: 15; Percent complete: 0.1%; Average loss: 5.6994
Iteration: 16; Percent complete: 0.2%

Iteration: 136; Percent complete: 1.4%; Average loss: 4.4349
Iteration: 137; Percent complete: 1.4%; Average loss: 4.2490
Iteration: 138; Percent complete: 1.4%; Average loss: 4.5179
Iteration: 139; Percent complete: 1.4%; Average loss: 4.5114
Iteration: 140; Percent complete: 1.4%; Average loss: 4.2065
Iteration: 141; Percent complete: 1.4%; Average loss: 4.3012
Iteration: 142; Percent complete: 1.4%; Average loss: 4.3909
Iteration: 143; Percent complete: 1.4%; Average loss: 4.3813
Iteration: 144; Percent complete: 1.4%; Average loss: 3.9729
Iteration: 145; Percent complete: 1.5%; Average loss: 4.3436
Iteration: 146; Percent complete: 1.5%; Average loss: 4.4216
Iteration: 147; Percent complete: 1.5%; Average loss: 4.0820
Iteration: 148; Percent complete: 1.5%; Average loss: 4.2664
Iteration: 149; Percent complete: 1.5%; Average loss: 4.2078
Iteration: 150; Percent complete: 1.5%; Average loss: 4.3410
Iteration: 151; Percent complete: 1.5%; Average loss: 4.1629
Iteration: 152; Percent 

Iteration: 271; Percent complete: 2.7%; Average loss: 4.0267
Iteration: 272; Percent complete: 2.7%; Average loss: 4.0934
Iteration: 273; Percent complete: 2.7%; Average loss: 3.9841
Iteration: 274; Percent complete: 2.7%; Average loss: 4.0797
Iteration: 275; Percent complete: 2.8%; Average loss: 3.5476
Iteration: 276; Percent complete: 2.8%; Average loss: 4.0972
Iteration: 277; Percent complete: 2.8%; Average loss: 3.8968
Iteration: 278; Percent complete: 2.8%; Average loss: 3.8789
Iteration: 279; Percent complete: 2.8%; Average loss: 4.0199
Iteration: 280; Percent complete: 2.8%; Average loss: 3.8944
Iteration: 281; Percent complete: 2.8%; Average loss: 4.0120
Iteration: 282; Percent complete: 2.8%; Average loss: 4.1061
Iteration: 283; Percent complete: 2.8%; Average loss: 3.9433
Iteration: 284; Percent complete: 2.8%; Average loss: 3.9154
Iteration: 285; Percent complete: 2.9%; Average loss: 3.8362
Iteration: 286; Percent complete: 2.9%; Average loss: 4.0925
Iteration: 287; Percent 

Iteration: 406; Percent complete: 4.1%; Average loss: 4.0923
Iteration: 407; Percent complete: 4.1%; Average loss: 3.7281
Iteration: 408; Percent complete: 4.1%; Average loss: 3.9286
Iteration: 409; Percent complete: 4.1%; Average loss: 3.7877
Iteration: 410; Percent complete: 4.1%; Average loss: 3.7680
Iteration: 411; Percent complete: 4.1%; Average loss: 3.8700
Iteration: 412; Percent complete: 4.1%; Average loss: 4.0140
Iteration: 413; Percent complete: 4.1%; Average loss: 3.8494
Iteration: 414; Percent complete: 4.1%; Average loss: 3.6163
Iteration: 415; Percent complete: 4.2%; Average loss: 3.9529
Iteration: 416; Percent complete: 4.2%; Average loss: 3.9271
Iteration: 417; Percent complete: 4.2%; Average loss: 3.6452
Iteration: 418; Percent complete: 4.2%; Average loss: 4.1294
Iteration: 419; Percent complete: 4.2%; Average loss: 3.8388
Iteration: 420; Percent complete: 4.2%; Average loss: 3.9909
Iteration: 421; Percent complete: 4.2%; Average loss: 3.9439
Iteration: 422; Percent 

Iteration: 543; Percent complete: 5.4%; Average loss: 3.9016
Iteration: 544; Percent complete: 5.4%; Average loss: 3.7352
Iteration: 545; Percent complete: 5.5%; Average loss: 3.4373
Iteration: 546; Percent complete: 5.5%; Average loss: 3.6931
Iteration: 547; Percent complete: 5.5%; Average loss: 3.5482
Iteration: 548; Percent complete: 5.5%; Average loss: 3.6488
Iteration: 549; Percent complete: 5.5%; Average loss: 3.7757
Iteration: 550; Percent complete: 5.5%; Average loss: 3.6745
Iteration: 551; Percent complete: 5.5%; Average loss: 3.7396
Iteration: 552; Percent complete: 5.5%; Average loss: 3.8420
Iteration: 553; Percent complete: 5.5%; Average loss: 3.7830
Iteration: 554; Percent complete: 5.5%; Average loss: 3.7596
Iteration: 555; Percent complete: 5.5%; Average loss: 3.5721
Iteration: 556; Percent complete: 5.6%; Average loss: 3.4535
Iteration: 557; Percent complete: 5.6%; Average loss: 3.6239
Iteration: 558; Percent complete: 5.6%; Average loss: 3.6622
Iteration: 559; Percent 

...

Iteration: 9391; Percent complete: 93.9%; Average loss: 1.4516
Iteration: 9392; Percent complete: 93.9%; Average loss: 1.5128
Iteration: 9393; Percent complete: 93.9%; Average loss: 1.5490
Iteration: 9394; Percent complete: 93.9%; Average loss: 1.5907
Iteration: 9395; Percent complete: 94.0%; Average loss: 1.7368
Iteration: 9396; Percent complete: 94.0%; Average loss: 1.6858
Iteration: 9397; Percent complete: 94.0%; Average loss: 1.4465
Iteration: 9398; Percent complete: 94.0%; Average loss: 1.6364
Iteration: 9399; Percent complete: 94.0%; Average loss: 1.6617
Iteration: 9400; Percent complete: 94.0%; Average loss: 1.5954
Iteration: 9401; Percent complete: 94.0%; Average loss: 1.3711
Iteration: 9402; Percent complete: 94.0%; Average loss: 1.6918
Iteration: 9403; Percent complete: 94.0%; Average loss: 1.8647
Iteration: 9404; Percent complete: 94.0%; Average loss: 1.6880
Iteration: 9405; Percent complete: 94.0%; Average loss: 1.5576
Iteration: 9406; Percent complete: 94.1%; Average loss:

Iteration: 9522; Percent complete: 95.2%; Average loss: 1.6884
Iteration: 9523; Percent complete: 95.2%; Average loss: 1.5929
Iteration: 9524; Percent complete: 95.2%; Average loss: 1.6559
Iteration: 9525; Percent complete: 95.2%; Average loss: 1.6449
Iteration: 9526; Percent complete: 95.3%; Average loss: 1.6073
Iteration: 9527; Percent complete: 95.3%; Average loss: 1.5663
Iteration: 9528; Percent complete: 95.3%; Average loss: 1.5244
Iteration: 9529; Percent complete: 95.3%; Average loss: 1.5480
Iteration: 9530; Percent complete: 95.3%; Average loss: 1.6869
Iteration: 9531; Percent complete: 95.3%; Average loss: 1.6471
Iteration: 9532; Percent complete: 95.3%; Average loss: 1.6094
Iteration: 9533; Percent complete: 95.3%; Average loss: 1.5099
Iteration: 9534; Percent complete: 95.3%; Average loss: 1.5014
Iteration: 9535; Percent complete: 95.3%; Average loss: 1.5721
Iteration: 9536; Percent complete: 95.4%; Average loss: 1.5678
Iteration: 9537; Percent complete: 95.4%; Average loss:

Iteration: 9654; Percent complete: 96.5%; Average loss: 1.5972
Iteration: 9655; Percent complete: 96.5%; Average loss: 1.4817
Iteration: 9656; Percent complete: 96.6%; Average loss: 1.5453
Iteration: 9657; Percent complete: 96.6%; Average loss: 1.5881
Iteration: 9658; Percent complete: 96.6%; Average loss: 1.5630
Iteration: 9659; Percent complete: 96.6%; Average loss: 1.6266
Iteration: 9660; Percent complete: 96.6%; Average loss: 1.8187
Iteration: 9661; Percent complete: 96.6%; Average loss: 1.7424
Iteration: 9662; Percent complete: 96.6%; Average loss: 1.6450
Iteration: 9663; Percent complete: 96.6%; Average loss: 1.6088
Iteration: 9664; Percent complete: 96.6%; Average loss: 1.3644
Iteration: 9665; Percent complete: 96.7%; Average loss: 1.5458
Iteration: 9666; Percent complete: 96.7%; Average loss: 1.6691
Iteration: 9667; Percent complete: 96.7%; Average loss: 1.3114
Iteration: 9668; Percent complete: 96.7%; Average loss: 1.5532
Iteration: 9669; Percent complete: 96.7%; Average loss:

Iteration: 9787; Percent complete: 97.9%; Average loss: 1.6250
Iteration: 9788; Percent complete: 97.9%; Average loss: 1.4406
Iteration: 9789; Percent complete: 97.9%; Average loss: 1.5331
Iteration: 9790; Percent complete: 97.9%; Average loss: 1.8846
Iteration: 9791; Percent complete: 97.9%; Average loss: 1.4708
Iteration: 9792; Percent complete: 97.9%; Average loss: 1.6756
Iteration: 9793; Percent complete: 97.9%; Average loss: 1.6306
Iteration: 9794; Percent complete: 97.9%; Average loss: 1.4998
Iteration: 9795; Percent complete: 98.0%; Average loss: 1.6424
Iteration: 9796; Percent complete: 98.0%; Average loss: 1.4100
Iteration: 9797; Percent complete: 98.0%; Average loss: 1.5165
Iteration: 9798; Percent complete: 98.0%; Average loss: 1.6758
Iteration: 9799; Percent complete: 98.0%; Average loss: 1.4214
Iteration: 9800; Percent complete: 98.0%; Average loss: 1.6690
Iteration: 9801; Percent complete: 98.0%; Average loss: 1.6755
Iteration: 9802; Percent complete: 98.0%; Average loss:

Iteration: 9919; Percent complete: 99.2%; Average loss: 1.4692
Iteration: 9920; Percent complete: 99.2%; Average loss: 1.6452
Iteration: 9921; Percent complete: 99.2%; Average loss: 1.4841
Iteration: 9922; Percent complete: 99.2%; Average loss: 1.3486
Iteration: 9923; Percent complete: 99.2%; Average loss: 1.3693
Iteration: 9924; Percent complete: 99.2%; Average loss: 1.2982
Iteration: 9925; Percent complete: 99.2%; Average loss: 1.5135
Iteration: 9926; Percent complete: 99.3%; Average loss: 1.3986
Iteration: 9927; Percent complete: 99.3%; Average loss: 1.5065
Iteration: 9928; Percent complete: 99.3%; Average loss: 1.5064
Iteration: 9929; Percent complete: 99.3%; Average loss: 1.7666
Iteration: 9930; Percent complete: 99.3%; Average loss: 1.5292
Iteration: 9931; Percent complete: 99.3%; Average loss: 1.4727
Iteration: 9932; Percent complete: 99.3%; Average loss: 1.5890
Iteration: 9933; Percent complete: 99.3%; Average loss: 1.5067
Iteration: 9934; Percent complete: 99.3%; Average loss:

In [19]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> hi
Bot: hi . . . . .
> hello
Bot: hello . . . . .
> how ARE YOU
Bot: fine . . . and go .
> WHAT DO YOU do
Bot: i m a comedian . .
> when did you became one
Bot: this morning . . . .
> is it really possible
Bot: is it really your leg ? ? ?
> trying not to laugh
Bot: what is it ? ? ? ?
> just chill and have a lot oif fun
Error: Encountered unknown word.
> lot of fun
Bot: did you hear what he said ? ?
> wht?
Error: Encountered unknown word.
> what
Bot: you re a good man player . .
> what
Bot: you re a good man player . .
> when
Bot: tonight . before i saw you .
> when
Bot: tonight . before i saw you .
> how is it possible
Bot: i don t know . and .
> when did you react
Error: Encountered unknown word.
> please
Bot: i m gonna go to work there . .
> go
Bot: what ? ? ? ? ?
> go baby go
Bot: what ? ? ? ? ?
> jiim
Error: Encountered unknown word.
> jim
Bot: yes sir . and falling . .
> love you
Bot: do you love me ? ?
> sure love
Bot: i m thinking about this .
> create risc
Error: Encountered 

KeyboardInterrupt: 